In [1]:
import pandas as pd
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlc = SQLContext(sc)

muertes_policias = sqlc.read.csv("muertes.csv",header=True).rdd
muertes_policias = muertes_policias.map(lambda x:(x[0],pd.to_datetime(x[1],"coerce"),x[2],x[3],x[4])).cache()

porcentaje_pobreza = sqlc.read.csv("pobreza.csv",header=True).rdd
porcentaje_pobreza = porcentaje_pobreza.map(lambda x:(x[0],x[1],pd.to_numeric(x[2],"coerce"))).cache()

In [3]:
#muertes_policias: (name, date, race, city, state)
#porcentaje_pobreza: (state, city, poverty_rate)

In [4]:
#A:
porcentaje_pobreza_estado = porcentaje_pobreza.map(lambda x:(x[0],(x[2],1))).\
        reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).\
        map(lambda x:(x[0],x[1][0]/x[1][1])).cache()#(estado,porcentaje prom)
porcentaje_pobreza_ciudad = porcentaje_pobreza.map(lambda x:(x[0],(x[1],x[2])))#(estado,(ciudad,pobreza))
porcentaje_pobreza_ciudad.join(porcentaje_pobreza_estado).\
        takeOrdered(10,lambda x:-__builtin__.abs(x[1][0][1]-x[1][1]))

[('S3', (('Washington DC', 0.95), 0.4483333333333333)),
 ('S3', (('San Francisco', 0.89), 0.4483333333333333)),
 ('S3', (('New York', 0.1), 0.4483333333333333)),
 ('S1', (('New Jersey', 0.14), 0.44833333333333325)),
 ('S3', (('New Jersey', 0.15), 0.4483333333333333)),
 ('S2', (('New York', 0.65), 0.36666666666666664)),
 ('S2', (('Washington DC', 0.09), 0.36666666666666664)),
 ('S2', (('San Francisco', 0.11), 0.36666666666666664)),
 ('S1', (('San Francisco', 0.7), 0.44833333333333325)),
 ('S3', (('Chicago', 0.2), 0.4483333333333333))]

In [5]:
#B
porcentaje_redondeado = porcentaje_pobreza_estado.\
        map(lambda x:(x[0],round(x[1],2)))#(estado, porcentaje prom redondeado)
muertes_negras = muertes_policias.filter(lambda x:x[2]=="Black").map(lambda x:(x[4],1))#(estado,1)
porcentaje_redondeado.join(muertes_negras).\
        map(lambda x:(x[1][0],x[1][1])).\
        reduceByKey(lambda x,y:x+y).collect()#(porcentaje pobreza, cantidad de muertes)

[(0.37, 68), (0.45, 125)]